# Lab Exercise: High-Frequency Arbitrage in Fragmented Markets

**Deadline:** 9th of December 23:59 CET

**Submission:** Email to francisco.merlos@six-group.com with title: "Arbitrage study in BME | Your name"

## 1. Context: The Fragmented Market

In modern European equity markets, liquidity is **fragmented**. The same stock (ISIN) trades simultaneously on the primary exchange (BME) and various Multilateral Trading Facilities (MTFs) like CBOE, Turquoise, and Aquis.

Due to this fragmentation, temporary price discrepancies occur. A stock might be offered for sale at €10.00 on Turquoise while a buyer is bidding €10.01 on BME. A High-Frequency Trader (HFT) can profit from this by buying low and selling high instantaneously.

However, these opportunities are fleeting. The "theoretical" profit you see in a snapshot might disappear by the time your order reaches the exchange due to **latency**.

### The Mission

You have been hired as a Quantitative Researcher at a proprietary trading firm. Your boss has given you a dataset of high-resolution market data and asked you to answer three critical questions:

1. **Do arbitrage opportunities still exist in Spanish equities?**
2. **What is the maximum theoretical profit** (assuming 0 latency)?
3. **The "Latency Decay" Curve:** How quickly does this profit vanish as our trading system gets slower (from 0µs to 100ms)?

## 2. Data Specifications

You are provided with a `DATA_BIG/` folder containing subfolders for specific trading dates. Inside, you will find three types of compressed CSV files for various instruments.

**Note:** You can also find a `DATA_SMALL` folder that you can use to test quickly without needing to run the simulation over all the data.

### File Naming Convention

The naming pattern for all three file types (QTE, STS, TRD) is:

```
<type>_<session>_<isin>_<ticker>_<mic>_<part>.csv.gz
```

| Field | Description |
|-------|-------------|
| **type** | QTE, TRD, or STS |
| **session** | Trading date (YYYY-MM-DD) |
| **isin** | Cross-venue **ISIN** (International Securities Identification Number) |
| **ticker** | Venue-specific trading symbol (distinguishes multiple books for the same ISIN on the same MIC) |
| **mic** | Market Identifier Code (MIC, e.g., XMAD) |
| **part** | Integer part number. Assume it is always 1 for simplicity. |

### Order Book Identity and Join Key

A single **order book identity** is defined by the tuple:

```
(session, isin, mic, ticker)
```

This identity is the **key used to join** corresponding QTE, TRD, and STS data belonging to the same book.

### File Types

1. **QTE (Quotes/Snapshots):** Represents the state of the order book (up to 10 levels deep).
   - `epoch`: Timestamp in microseconds (UTC).
   - `px_bid_0`, `px_ask_0`: Best Bid and Best Ask prices.
   - `qty_bid_0`, `qty_ask_0`: Available volume at the best price.
   - *Note: Columns exist for levels 0-9.*

2. **STS (Trading Status):** Updates on the market phase (e.g., Open, Auction, Closed).
   - `epoch`: Timestamp.
   - `market_trading_status`: An integer code representing the state.

3. **TRD (Trades):** Represents the transactions. Not needed for this exercise.

### CRITICAL: Vendor Data Definitions

Real-world financial data is rarely clean. The data vendor has provided the following specifications. **Ignoring these will result in massive errors in your P&L calculation.**

#### A. "Magic Numbers" (Special Price Codes)

The vendor uses specific high-value constants to indicate non-tradable states (e.g., Market Orders during auctions). **These are NOT real prices.** If you treat 999,999 as a valid bid, your algorithm will assume you can sell for a million euros.

| Value | Meaning | Action Required |
|-------|---------|----------------|
| 666666.666 | Unquoted/Unknown | **Discard** |
| 999999.999 | Market Order (At Best) | **Discard** |
| 999999.989 | At Open Order | **Discard** |
| 999999.988 | At Close Order | **Discard** |
| 999999.979 | Pegged Order | **Discard** |
| 999999.123 | Unquoted/Unknown | **Discard** |

#### B. Market Status Codes

You can only trade when the market is in **Continuous Trading**. If you trade during an Auction, a Halt, or Pre-Open, your order will not execute immediately. A snapshot is only valid/addressable if the STS for that venue is one of these codes:

| Venue | Continuous Trading Code |
|-------|------------------------|
| AQUIS | 5308427 |
| BME | 5832713, 5832756 |
| CBOE | 12255233 |
| TURQUOISE | 7608181 |

## 3. Implementation Guide

You are encouraged to use AI tools (ChatGPT, Claude, etc.) to generate the Python/Pandas code. However, **you** are responsible for the logic and the financial validity of the results.

### Step 1: Data Ingestion & Cleaning

- Write a function to load the QTE and STS files for a given ISIN.
- **Task:** Ensure you are using only valid prices
- **Task:** Ensure you are only looking at addressable orderbooks

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
from pathlib import Path
import warnings

# ============================================================================
# STEP 1: DATA INGESTION & CLEANING
# ============================================================================

# Define magic numbers that represent invalid/non-tradable prices
# These are special codes used by the vendor to indicate non-tradable states
# (e.g., Market Orders during auctions). Treating these as real prices would
# cause massive errors in P&L calculations.
MAGIC_NUMBERS = [
    666666.666,  # Unquoted/Unknown
    999999.999,  # Market Order (At Best)
    999999.989,  # At Open Order
    999999.988,  # At Close Order
    999999.979,  # Pegged Order
    999999.123   # Unquoted/Unknown
]

# Define continuous trading status codes per venue
# These codes indicate when the market is in continuous trading mode,
# which is the only time orders can execute immediately. Trading during
# auctions, halts, or pre-open would not execute immediately.
CONTINUOUS_TRADING_STATUS = {
    'AQEU': [5308427],           # AQUIS
    'XMAD': [5832713, 5832756],  # BME (two codes for different segments)
    'CEUX': [12255233],          # CBOE
    'TQEX': [7608181]            # TURQUOISE
}


def discover_files_for_isin(isin, data_path='DATA_BIG'):
    """
    Discover all QTE and STS files for a given ISIN across all venues and dates.
    
    This function scans the data directory structure to find all relevant files
    for the specified ISIN. It handles the file naming convention:
    <type>_<session>_<isin>_<ticker>_<mic>_<part>.csv.gz
    
    Parameters:
    -----------
    isin : str
        The ISIN (International Securities Identification Number) to search for
    data_path : str
        Path to the data directory (default: 'DATA_BIG')
    
    Returns:
    --------
    dict
        Dictionary with keys 'qte' and 'sts', each containing a list of
        file paths grouped by orderbook identity (session, isin, mic, ticker)
    """
    qte_files = {}
    sts_files = {}
    
    # Get all venue-date folders (e.g., BME_2025-11-07, CBOE_2025-11-07)
    data_dir = Path(data_path)
    if not data_dir.exists():
        raise ValueError(f"Data path does not exist: {data_path}")
    
    # Find all venue-date folders
    venue_date_folders = [d for d in data_dir.iterdir() if d.is_dir()]
    
    # Search for QTE and STS files matching the ISIN pattern
    for folder in venue_date_folders:
        # Pattern: QTE_*_<isin>_*.csv.gz or STS_*_<isin>_*.csv.gz
        qte_pattern = str(folder / f'QTE_*_{isin}_*.csv.gz')
        sts_pattern = str(folder / f'STS_*_{isin}_*.csv.gz')
        
        # Find matching files
        qte_matches = glob.glob(qte_pattern)
        sts_matches = glob.glob(sts_pattern)
        
        # Process QTE files
        for file_path in qte_matches:
            # Parse filename to extract orderbook identity
            # Format: QTE_<session>_<isin>_<ticker>_<mic>_<part>.csv.gz
            filename = Path(file_path).name
            parts = filename.replace('.csv.gz', '').split('_')
            
            if len(parts) >= 6:
                session = parts[1]
                file_isin = parts[2]
                ticker = parts[3]
                mic = parts[4]
                part = int(parts[5])
                
                # Create orderbook identity key
                identity = (session, file_isin, mic, ticker)
                
                # Handle multiple part numbers - keep only the highest part
                if identity in qte_files:
                    existing_part = int(Path(qte_files[identity]).name.split('_')[5].replace('.csv.gz', ''))
                    if part > existing_part:
                        warnings.warn(
                            f"Multiple part numbers found for {identity}. "
                            f"Using part {part} instead of {existing_part}."
                        )
                        qte_files[identity] = file_path
                    elif part < existing_part:
                        warnings.warn(
                            f"Multiple part numbers found for {identity}. "
                            f"Using part {existing_part} instead of {part}."
                        )
                else:
                    qte_files[identity] = file_path
        
        # Process STS files (same logic)
        for file_path in sts_matches:
            filename = Path(file_path).name
            parts = filename.replace('.csv.gz', '').split('_')
            
            if len(parts) >= 6:
                session = parts[1]
                file_isin = parts[2]
                ticker = parts[3]
                mic = parts[4]
                part = int(parts[5])
                
                identity = (session, file_isin, mic, ticker)
                
                if identity in sts_files:
                    existing_part = int(Path(sts_files[identity]).name.split('_')[5].replace('.csv.gz', ''))
                    if part > existing_part:
                        warnings.warn(
                            f"Multiple part numbers found for {identity}. "
                            f"Using part {part} instead of {existing_part}."
                        )
                        sts_files[identity] = file_path
                    elif part < existing_part:
                        warnings.warn(
                            f"Multiple part numbers found for {identity}. "
                            f"Using part {existing_part} instead of {part}."
                        )
                else:
                    sts_files[identity] = file_path
    
    return {'qte': qte_files, 'sts': sts_files}


def clean_qte_timestamps(df):
    """
    Clean and create unique timestamp index for QTE (orderbook snapshot) data.
    
    For orderbook snapshots, if multiple snapshots occur at the same microsecond,
    we only care about the final state (last snapshot). This is because the orderbook
    state at the end of that microsecond is what matters for trading decisions.
    
    After keeping the last snapshot per epoch, we apply the nanosecond trick to
    ensure unique timestamps for indexing purposes.
    
    Parameters:
    -----------
    df : pd.DataFrame
        QTE DataFrame with 'epoch' column
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with unique timestamp index
    """
    # Sort by epoch (and sequence if it exists)
    sort_cols = ['epoch']
    if 'sequence' in df.columns:
        sort_cols.append('sequence')
    df = df.sort_values(by=sort_cols, ascending=[True] * len(sort_cols))
    
    # For orderbook snapshots, keep only the last snapshot per epoch
    # Multiple snapshots at the same microsecond represent intermediate states,
    # but we only care about the final state for trading decisions
    df = df.drop_duplicates(subset='epoch', keep='last').copy()
    
    # Convert epoch to datetime (microseconds since Unix epoch)
    temp_ts = pd.to_datetime(df['epoch'], unit='us')
    
    # Apply nanosecond trick to handle any remaining duplicates
    # This creates a unique timestamp index by adding nanoseconds
    offset_ns = df.groupby('epoch').cumcount()
    
    if len(offset_ns) > 0 and offset_ns.max() > 1000:
        raise Exception(
            f"There are more than 1000 snapshots happening at the same microsecond. "
            f"Max number: {offset_ns.max()}"
        )
    
    # Create final high-resolution timestamp
    df['ts'] = temp_ts + pd.to_timedelta(offset_ns, unit='ns')
    
    # Set timestamp as index
    df.set_index('ts', inplace=True)
    
    return df


def clean_sts_timestamps(df):
    """
    Clean and create unique timestamp index for STS (trading status) data.
    
    Similar to QTE, but STS typically has fewer duplicates. We apply the
    nanosecond trick to ensure unique timestamps.
    
    Parameters:
    -----------
    df : pd.DataFrame
        STS DataFrame with 'epoch' column
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with unique timestamp index
    """
    # Sort by epoch
    df = df.sort_values(by='epoch', ascending=True)
    
    # Convert epoch to datetime
    temp_ts = pd.to_datetime(df['epoch'], unit='us')
    
    # Apply nanosecond trick for unique timestamps
    offset_ns = df.groupby('epoch').cumcount()
    
    if len(offset_ns) > 0 and offset_ns.max() > 1000:
        raise Exception(
            f"There are more than 1000 status updates at the same microsecond. "
            f"Max number: {offset_ns.max()}"
        )
    
    # Create final timestamp
    df['ts'] = temp_ts + pd.to_timedelta(offset_ns, unit='ns')
    
    # Set timestamp as index
    df.set_index('ts', inplace=True)
    
    return df


def load_qte_sts_for_isin(isin, data_path='DATA_BIG'):
    """
    Load and clean QTE (quotes) and STS (trading status) files for a given ISIN.
    
    This function:
    1. Discovers all QTE and STS files for the ISIN across all venues and dates
    2. Loads the data using pandas
    3. Filters out invalid prices (magic numbers)
    4. Filters for addressable orderbooks (continuous trading status only)
    5. Creates unique timestamp indexes using the nanosecond trick
    
    Parameters:
    -----------
    isin : str
        The ISIN to load data for
    data_path : str
        Path to the data directory (default: 'DATA_BIG')
    
    Returns:
    --------
    tuple
        (df_qte_clean, df_sts_clean) - Cleaned QTE and STS DataFrames
    """
    # Discover all files for this ISIN
    files = discover_files_for_isin(isin, data_path)
    
    qte_files = files['qte']
    sts_files = files['sts']
    
    if not qte_files and not sts_files:
        warnings.warn(f"No QTE or STS files found for ISIN: {isin}")
        return pd.DataFrame(), pd.DataFrame()
    
    # ========================================================================
    # LOAD AND CLEAN QTE FILES
    # ========================================================================
    qte_dataframes = []
    
    for identity, file_path in qte_files.items():
        session, file_isin, mic, ticker = identity
        
        # Load QTE file
        # Using pandas read_csv with compression='gzip' for efficient loading
        df_qte = pd.read_csv(file_path, sep=';', compression='gzip')
        
        if df_qte.empty:
            continue
        
        # Add orderbook identity columns for tracking
        df_qte['session'] = session
        df_qte['isin'] = file_isin
        df_qte['mic'] = mic
        df_qte['ticker'] = ticker
        
        # Filter invalid prices (magic numbers)
        # These represent non-tradable states and must be discarded
        # We check both bid and ask prices at level 0 (best prices)
        valid_mask = (
            ~df_qte['px_bid_0'].isin(MAGIC_NUMBERS) &
            ~df_qte['px_ask_0'].isin(MAGIC_NUMBERS)
        )
        df_qte = df_qte[valid_mask].copy()
        
        if df_qte.empty:
            warnings.warn(
                f"All prices filtered out (magic numbers) for {identity}. "
                f"Skipping this orderbook."
            )
            continue
        
        # Clean timestamps and create unique index
        df_qte = clean_qte_timestamps(df_qte)
        
        # Keep only essential columns for Step 1
        # We focus on best bid/ask (level 0) for arbitrage detection
        essential_cols = ['session', 'isin', 'mic', 'ticker', 
                         'px_bid_0', 'px_ask_0', 'qty_bid_0', 'qty_ask_0']
        df_qte = df_qte[essential_cols].copy()
        
        qte_dataframes.append(df_qte)
    
    # Concatenate all QTE DataFrames
    if qte_dataframes:
        df_qte_clean = pd.concat(qte_dataframes, axis=0)
        df_qte_clean.sort_index(inplace=True)
    else:
        df_qte_clean = pd.DataFrame()
        warnings.warn(f"No valid QTE data after filtering for ISIN: {isin}")
    
    # ========================================================================
    # LOAD AND CLEAN STS FILES
    # ========================================================================
    sts_dataframes = []
    
    for identity, file_path in sts_files.items():
        session, file_isin, mic, ticker = identity
        
        # Load STS file
        df_sts = pd.read_csv(file_path, sep=';', compression='gzip')
        
        if df_sts.empty:
            continue
        
        # Add orderbook identity columns
        df_sts['session'] = session
        df_sts['isin'] = file_isin
        df_sts['mic'] = mic
        df_sts['ticker'] = ticker
        
        # Filter for addressable orderbooks (continuous trading status only)
        # Only orders placed during continuous trading can execute immediately
        # Trading during auctions, halts, or pre-open would not execute
        if mic in CONTINUOUS_TRADING_STATUS:
            valid_statuses = CONTINUOUS_TRADING_STATUS[mic]
            addressable_mask = df_sts['market_trading_status'].isin(valid_statuses)
            df_sts = df_sts[addressable_mask].copy()
        else:
            # If MIC not in our mapping, warn and skip filtering
            warnings.warn(
                f"Unknown MIC '{mic}' for {identity}. "
                f"Cannot filter by trading status. Including all statuses."
            )
        
        if df_sts.empty:
            warnings.warn(
                f"No addressable orderbook periods (continuous trading) for {identity}. "
                f"Skipping this orderbook."
            )
            continue
        
        # Clean timestamps and create unique index
        df_sts = clean_sts_timestamps(df_sts)
        
        # Keep essential columns
        essential_cols = ['session', 'isin', 'mic', 'ticker', 'market_trading_status']
        df_sts = df_sts[essential_cols].copy()
        
        sts_dataframes.append(df_sts)
    
    # Concatenate all STS DataFrames
    if sts_dataframes:
        df_sts_clean = pd.concat(sts_dataframes, axis=0)
        df_sts_clean.sort_index(inplace=True)
    else:
        df_sts_clean = pd.DataFrame()
        warnings.warn(f"No valid STS data after filtering for ISIN: {isin}")
    
    return df_qte_clean, df_sts_clean


# ============================================================================
# TEST THE FUNCTION
# ============================================================================
# Test with a sample ISIN from DATA_SMALL first to verify the function works
# Then we can use it with DATA_BIG

print("Testing load_qte_sts_for_isin function...")
print("=" * 80)

# Test with sample ISIN from DATA_SMALL
test_isin = 'ES0113900J37'
df_qte, df_sts = load_qte_sts_for_isin(test_isin, data_path='DATA_SMALL')

print(f"\nQTE DataFrame shape: {df_qte.shape}")
print(f"QTE columns: {df_qte.columns.tolist() if not df_qte.empty else 'Empty'}")
if not df_qte.empty:
    print(f"QTE date range: {df_qte.index.min()} to {df_qte.index.max()}")
    print(f"\nQTE sample (first 5 rows):")
    print(df_qte.head())

print(f"\nSTS DataFrame shape: {df_sts.shape}")
print(f"STS columns: {df_sts.columns.tolist() if not df_sts.empty else 'Empty'}")
if not df_sts.empty:
    print(f"STS date range: {df_sts.index.min()} to {df_sts.index.max()}")
    print(f"\nSTS sample (first 5 rows):")
    print(df_sts.head())

print("\n" + "=" * 80)
print("Function implementation complete!")

### Step 2: Create the "Consolidated Tape"

- To detect arbitrage, you need to compare prices across venues *at the exact same time*.
- **Task:** Create a single DataFrame where the index is the timestamp, and the columns represent the Best Bid and Best Ask for **every** venue (BME, XMAD, CBOE, etc.).

In [ ]:
# ============================================================================
# STEP 2: CREATE THE CONSOLIDATED TAPE
# ============================================================================

def create_consolidated_tape(df_qte, df_sts):
    """
    Create a consolidated tape showing best bid and ask prices for all venues over time.
    
    This function:
    1. Merges QTE snapshots with STS status to ensure only addressable orderbooks
    2. Creates separate pivots for bid and ask prices per venue
    3. Forward-fills prices to handle asynchronous updates across venues
    4. Returns a unified view where each timestamp shows all venues' best bid/ask
    
    The consolidated tape is essential for arbitrage detection because it allows
    us to compare prices across venues at the exact same moment in time.
    
    Parameters:
    -----------
    df_qte : pd.DataFrame
        Cleaned QTE DataFrame from load_qte_sts_for_isin()
        Must have columns: timestamp index, 'session', 'isin', 'mic', 'ticker',
        'px_bid_0', 'px_ask_0', 'qty_bid_0', 'qty_ask_0'
    df_sts : pd.DataFrame
        Cleaned STS DataFrame from load_qte_sts_for_isin()
        Must have columns: timestamp index, 'session', 'isin', 'mic', 'ticker',
        'market_trading_status'
    
    Returns:
    --------
    pd.DataFrame
        Consolidated tape with timestamp index and columns:
        - {MIC}_bid, {MIC}_ask for each venue (e.g., XMAD_bid, XMAD_ask, CEUX_bid, CEUX_ask)
        - Forward-filled prices (last known price propagates forward)
    """
    if df_qte.empty or df_sts.empty:
        warnings.warn("Empty QTE or STS DataFrame provided. Returning empty consolidated tape.")
        return pd.DataFrame()
    
    # ========================================================================
    # STEP 1: MERGE QTE WITH STS PER ORDERBOOK
    # ========================================================================
    # We need to ensure each QTE snapshot is only included when the market
    # is in continuous trading (addressable). We use merge_asof to find the
    # most recent STS status before/at each QTE snapshot.
    
    qte_filtered_list = []
    
    # Get unique orderbook identities
    qte_identities = df_qte[['session', 'isin', 'mic', 'ticker']].drop_duplicates()
    sts_identities = df_sts[['session', 'isin', 'mic', 'ticker']].drop_duplicates()
    
    # Process each orderbook identity
    for _, identity_row in qte_identities.iterrows():
        session = identity_row['session']
        isin = identity_row['isin']
        mic = identity_row['mic']
        ticker = identity_row['ticker']
        
        identity = (session, isin, mic, ticker)
        
        # Get QTE data for this orderbook
        qte_mask = (
            (df_qte['session'] == session) &
            (df_qte['isin'] == isin) &
            (df_qte['mic'] == mic) &
            (df_qte['ticker'] == ticker)
        )
        qte_orderbook = df_qte[qte_mask].copy()
        
        if qte_orderbook.empty:
            continue
        
        # Get STS data for this orderbook
        sts_mask = (
            (df_sts['session'] == session) &
            (df_sts['isin'] == isin) &
            (df_sts['mic'] == mic) &
            (df_sts['ticker'] == ticker)
        )
        sts_orderbook = df_sts[sts_mask].copy()
        
        if sts_orderbook.empty:
            # No STS data means we can't verify addressability - skip this orderbook
            warnings.warn(
                f"No STS data for orderbook {identity}. "
                f"Cannot verify addressability. Skipping."
            )
            continue
        
        # Prepare for merge_asof: reset index to make timestamp a column
        qte_for_merge = qte_orderbook.reset_index()
        sts_for_merge = sts_orderbook.reset_index()
        
        # Ensure both are sorted by timestamp (required for merge_asof)
        qte_for_merge = qte_for_merge.sort_values('ts')
        sts_for_merge = sts_for_merge.sort_values('ts')
        
        # Merge QTE with STS using merge_asof
        # direction='backward' means: for each QTE snapshot, find the most recent
        # STS status that occurred at or before that snapshot's timestamp.
        # This ensures we only use status information that was known at the time.
        qte_with_status = pd.merge_asof(
            qte_for_merge,
            sts_for_merge[['ts', 'market_trading_status']],
            on='ts',
            direction='backward'  # Look backwards in time (no look-ahead bias)
        )
        
        # Filter to keep only snapshots where market is in continuous trading
        # This ensures we only include addressable orderbooks
        if mic in CONTINUOUS_TRADING_STATUS:
            valid_statuses = CONTINUOUS_TRADING_STATUS[mic]
            addressable_mask = qte_with_status['market_trading_status'].isin(valid_statuses)
            qte_addressable = qte_with_status[addressable_mask].copy()
        else:
            # Unknown MIC - warn but include all data
            warnings.warn(
                f"Unknown MIC '{mic}' for {identity}. "
                f"Cannot filter by trading status. Including all snapshots."
            )
            qte_addressable = qte_with_status.copy()
        
        if qte_addressable.empty:
            continue
        
        # Set timestamp back as index
        qte_addressable.set_index('ts', inplace=True)
        
        # Keep only essential columns
        essential_cols = ['session', 'isin', 'mic', 'ticker',
                         'px_bid_0', 'px_ask_0', 'qty_bid_0', 'qty_ask_0']
        qte_addressable = qte_addressable[essential_cols].copy()
        
        qte_filtered_list.append(qte_addressable)
    
    if not qte_filtered_list:
        warnings.warn("No addressable QTE data after merging with STS. Returning empty tape.")
        return pd.DataFrame()
    
    # ========================================================================
    # STEP 2: COMBINE ALL VENUES
    # ========================================================================
    # Concatenate all filtered QTE data from all orderbooks
    # This creates a single DataFrame with all venues' snapshots
    qte_combined = pd.concat(qte_filtered_list, axis=0)
    qte_combined.sort_index(inplace=True)
    
    # ========================================================================
    # STEP 3: CREATE BID PRICE PIVOT
    # ========================================================================
    # Pivot the data to create columns for each venue's bid price
    # This transforms from long format (one row per snapshot) to wide format
    # (one row per timestamp, one column per venue)
    
    # Reset index temporarily to use timestamp as a column for pivot
    qte_reset = qte_combined.reset_index()
    
    # Create pivot table: columns = venue (MIC), values = bid price
    # This creates a DataFrame where each row is a timestamp and each column
    # is a venue's bid price
    bid_pivot = qte_reset.pivot(index='ts', columns='mic', values='px_bid_0')
    
    # Forward fill: propagate the last known bid price forward
    # This handles the asynchronous nature of market data - if a venue doesn't
    # update at a given timestamp, we use its last known price
    # This is the "Last Traded Price" (LTP) logic: if no update happened,
    # the valid price is still the last price that did happen
    bid_pivot_ffilled = bid_pivot.ffill()
    
    # Rename columns to indicate these are bid prices
    bid_pivot_ffilled.columns = [f'{mic}_bid' for mic in bid_pivot_ffilled.columns]
    
    # ========================================================================
    # STEP 4: CREATE ASK PRICE PIVOT
    # ========================================================================
    # Same process for ask prices
    ask_pivot = qte_reset.pivot(index='ts', columns='mic', values='px_ask_0')
    ask_pivot_ffilled = ask_pivot.ffill()
    
    # Rename columns to indicate these are ask prices
    ask_pivot_ffilled.columns = [f'{mic}_ask' for mic in ask_pivot_ffilled.columns]
    
    # ========================================================================
    # STEP 5: COMBINE BID AND ASK INTO FINAL TAPE
    # ========================================================================
    # Merge the bid and ask pivots on timestamp index
    # This creates the final consolidated tape with both bid and ask prices
    # for each venue at each timestamp
    
    # Use outer join to include all timestamps from both bid and ask
    consolidated_tape = pd.merge(
        bid_pivot_ffilled,
        ask_pivot_ffilled,
        left_index=True,
        right_index=True,
        how='outer'
    )
    
    # Sort by timestamp to ensure chronological order
    consolidated_tape.sort_index(inplace=True)
    
    # Forward fill one more time after merging to handle any remaining gaps
    # This ensures that if a venue has a bid but no ask (or vice versa) at
    # a given timestamp, we still have the last known value
    consolidated_tape = consolidated_tape.ffill()
    
    return consolidated_tape


# ============================================================================
# TEST THE FUNCTION
# ============================================================================
# Test with the sample data from Step 1

print("Testing create_consolidated_tape function...")
print("=" * 80)

# Use the test data from Step 1 (if it exists)
if 'df_qte' in locals() and 'df_sts' in locals() and not df_qte.empty and not df_sts.empty:
    consolidated_tape = create_consolidated_tape(df_qte, df_sts)
    
    print(f"\nConsolidated Tape shape: {consolidated_tape.shape}")
    print(f"Consolidated Tape columns: {consolidated_tape.columns.tolist()}")
    if not consolidated_tape.empty:
        print(f"Date range: {consolidated_tape.index.min()} to {consolidated_tape.index.max()}")
        print(f"\nConsolidated Tape sample (first 10 rows):")
        print(consolidated_tape.head(10))
        
        # Show summary statistics
        print(f"\nSummary Statistics:")
        print(consolidated_tape.describe())
        
        # Show which venues are present
        venues_bid = [col.replace('_bid', '') for col in consolidated_tape.columns if col.endswith('_bid')]
        venues_ask = [col.replace('_ask', '') for col in consolidated_tape.columns if col.endswith('_ask')]
        print(f"\nVenues with bid data: {venues_bid}")
        print(f"Venues with ask data: {venues_ask}")
else:
    print("\nNote: Run Step 1 first to load QTE and STS data, then run this cell again.")
    print("Or test with:")
    print("  test_isin = 'ES0113900J37'")
    print("  df_qte, df_sts = load_qte_sts_for_isin(test_isin, data_path='DATA_SMALL')")
    print("  consolidated_tape = create_consolidated_tape(df_qte, df_sts)")

print("\n" + "=" * 80)
print("Consolidated tape function implementation complete!")


# ============================================================================
# VISUALIZATIONS: SHOW DATA ACROSS ALL VENUES
# ============================================================================
# Similar to the reference notebook, we create visualizations to compare
# prices across all venues. This helps identify arbitrage opportunities
# and understand market microstructure dynamics.

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Set style for better-looking charts
plt.style.use('seaborn-v0_8-darkgrid' if 'seaborn-v0_8-darkgrid' in plt.style.available else 'default')

# Only create visualizations if we have consolidated tape data
if 'consolidated_tape' in locals() and not consolidated_tape.empty:
    print("\n" + "=" * 80)
    print("Creating visualizations of consolidated tape data...")
    print("=" * 80)
    
    # Get list of venues dynamically from the consolidated tape
    bid_cols = [col for col in consolidated_tape.columns if col.endswith('_bid')]
    ask_cols = [col for col in consolidated_tape.columns if col.endswith('_ask')]
    venues = sorted(set([col.replace('_bid', '').replace('_ask', '') for col in bid_cols + ask_cols]))
    
    if venues:
        # ========================================================================
        # CHART 1: BID AND ASK PRICES ACROSS ALL VENUES
        # ========================================================================
        # This chart shows the best bid and ask prices for each venue over time.
        # It allows us to visually identify when prices diverge across venues,
        # which is essential for spotting arbitrage opportunities.
        
        fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
        
        # Define colors and line styles for different venues
        # Using a consistent color scheme that works well for financial data
        colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
        line_styles = ['-', '--', '-.', ':', '-', '--']
        
        # Plot Bid Prices (top subplot)
        ax1 = axes[0]
        for i, venue in enumerate(venues):
            bid_col = f'{venue}_bid'
            if bid_col in consolidated_tape.columns:
                ax1.plot(
                    consolidated_tape.index,
                    consolidated_tape[bid_col],
                    label=f'{venue} Bid',
                    color=colors[i % len(colors)],
                    linestyle=line_styles[i % len(line_styles)],
                    linewidth=1.5,
                    alpha=0.8
                )
        
        ax1.set_ylabel('Bid Price (€)', fontsize=12, fontweight='bold')
        ax1.set_title('Best Bid Prices Across All Venues Over Time', fontsize=14, fontweight='bold')
        ax1.legend(loc='best', fontsize=10)
        ax1.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
        
        # Plot Ask Prices (bottom subplot)
        ax2 = axes[1]
        for i, venue in enumerate(venues):
            ask_col = f'{venue}_ask'
            if ask_col in consolidated_tape.columns:
                ax2.plot(
                    consolidated_tape.index,
                    consolidated_tape[ask_col],
                    label=f'{venue} Ask',
                    color=colors[i % len(colors)],
                    linestyle=line_styles[i % len(line_styles)],
                    linewidth=1.5,
                    alpha=0.8
                )
        
        ax2.set_xlabel('Timestamp', fontsize=12, fontweight='bold')
        ax2.set_ylabel('Ask Price (€)', fontsize=12, fontweight='bold')
        ax2.set_title('Best Ask Prices Across All Venues Over Time', fontsize=14, fontweight='bold')
        ax2.legend(loc='best', fontsize=10)
        ax2.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
        
        # Format x-axis to show time nicely
        for ax in axes:
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
            ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=max(1, len(consolidated_tape) // 100)))
            plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        plt.tight_layout()
        plt.show()
        
        # ========================================================================
        # CHART 2: PRICE SPREADS BETWEEN VENUES
        # ========================================================================
        # Calculate and visualize price differences (spreads) between venues.
        # Positive spreads indicate potential arbitrage opportunities where
        # one venue's bid is higher than another venue's ask.
        
        if len(venues) >= 2:
            # Calculate spreads between the first two venues (most common case)
            # In a real scenario, you might want to calculate all pairwise spreads
            venue1 = venues[0]
            venue2 = venues[1] if len(venues) > 1 else None
            
            if venue2:
                bid_col1 = f'{venue1}_bid'
                ask_col2 = f'{venue2}_ask'
                
                if bid_col1 in consolidated_tape.columns and ask_col2 in consolidated_tape.columns:
                    # Calculate spread: Bid from venue1 - Ask from venue2
                    # Positive spread means we can buy on venue2 and sell on venue1
                    spread = consolidated_tape[bid_col1] - consolidated_tape[ask_col2]
                    
                    fig, ax = plt.subplots(figsize=(14, 6))
                    
                    ax.plot(
                        consolidated_tape.index,
                        spread,
                        label=f'Spread ({venue1} Bid - {venue2} Ask)',
                        color='#2ca02c',
                        linewidth=1.5,
                        alpha=0.8
                    )
                    
                    # Add horizontal line at zero for reference
                    ax.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Zero Spread')
                    
                    # Highlight positive spreads (arbitrage opportunities)
                    positive_spread = spread[spread > 0]
                    if not positive_spread.empty:
                        ax.fill_between(
                            consolidated_tape.index,
                            0,
                            spread,
                            where=(spread > 0),
                            color='green',
                            alpha=0.2,
                            label='Arbitrage Opportunity'
                        )
                    
                    ax.set_xlabel('Timestamp', fontsize=12, fontweight='bold')
                    ax.set_ylabel('Price Spread (€)', fontsize=12, fontweight='bold')
                    ax.set_title(
                        f'Price Spread Between Venues: {venue1} Bid vs {venue2} Ask\n'
                        f'(Positive values indicate arbitrage opportunities)',
                        fontsize=14,
                        fontweight='bold'
                    )
                    ax.legend(loc='best', fontsize=10)
                    ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
                    
                    # Format x-axis
                    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                    ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=max(1, len(consolidated_tape) // 100)))
                    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # Print summary statistics
                    print(f"\nSpread Statistics ({venue1} Bid - {venue2} Ask):")
                    print(f"  Mean spread: {spread.mean():.4f} €")
                    print(f"  Max spread: {spread.max():.4f} €")
                    print(f"  Min spread: {spread.min():.4f} €")
                    print(f"  Std deviation: {spread.std():.4f} €")
                    print(f"  Positive spread periods: {len(positive_spread)} ({len(positive_spread)/len(spread)*100:.2f}%)")
        
        # ========================================================================
        # CHART 3: COMBINED VIEW - ALL BID AND ASK PRICES TOGETHER
        # ========================================================================
        # This chart shows both bid and ask prices for all venues on the same plot.
        # It provides a comprehensive view of the market microstructure and helps
        # identify when the bid-ask spread widens or narrows across venues.
        
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Plot all bid prices
        for i, venue in enumerate(venues):
            bid_col = f'{venue}_bid'
            ask_col = f'{venue}_ask'
            
            if bid_col in consolidated_tape.columns:
                ax.plot(
                    consolidated_tape.index,
                    consolidated_tape[bid_col],
                    label=f'{venue} Bid',
                    color=colors[i % len(colors)],
                    linestyle='-',
                    linewidth=2,
                    alpha=0.7
                )
            
            if ask_col in consolidated_tape.columns:
                ax.plot(
                    consolidated_tape.index,
                    consolidated_tape[ask_col],
                    label=f'{venue} Ask',
                    color=colors[i % len(colors)],
                    linestyle='--',
                    linewidth=2,
                    alpha=0.7
                )
        
        ax.set_xlabel('Timestamp', fontsize=12, fontweight='bold')
        ax.set_ylabel('Price (€)', fontsize=12, fontweight='bold')
        ax.set_title(
            'Consolidated Tape: All Venues Bid and Ask Prices Over Time\n'
            '(Solid lines = Bid, Dashed lines = Ask)',
            fontsize=14,
            fontweight='bold'
        )
        ax.legend(loc='best', fontsize=9, ncol=2)
        ax.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
        
        # Format x-axis
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
        ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=max(1, len(consolidated_tape) // 100)))
        plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')
        
        plt.tight_layout()
        plt.show()
        
        print("\nVisualizations complete!")
        print("=" * 80)
    else:
        print("\nNo venue data available for visualization.")
else:
    print("\nNote: Run the consolidated tape creation code above first to generate visualizations.")


### Step 3: Signal Generation

- **Arbitrage Condition:** An opportunity exists when Global Max Bid > Global Min Ask.
- **Profit Calc:** (Max Bid - Min Ask) * Min(BidQty, AskQty).
- **Rising Edge:** In a simulation, if an opportunity persists for 1 second (1000 snapshots), you can only trade it *once* (the first time it appears). Ensure you aren't "double counting" the same opportunity. If the opportunity vanishes and quickly reappears you can count it as a new opportunity for simplification.
- **Simplification:** Only look at opportunities between Global Max Bid and Global Min Ask. There might be others at the second or third price levels of the orderbook, but let's make it simple and use only the best Bid Ask of each trading venue.

In [ ]:
# Your code here


### Step 4: The "Time Machine" (Latency Simulation)

- In reality, if you see a price at time $T$, you cannot trade until $T + \Delta$.
- **Task:** Simulate execution latencies of [0, 100, 500, 1000, 2000, 3000, 4000, 5000, 10000, 15000, 20000, 30000, 50000, 100000] microseconds
- *Method:* If a signal is detected at T, look up what the profit *actually is* at T + Latency in your dataframe.

In [ ]:
# Your code here


## 4. Deliverables & Evaluation

Submit a Jupyter Notebook containing your code and the following analysis:

1. **The "Money Table":** A summary table showing the Total Realized Profit for all processed ISINs at each latency level.
2. **The Decay Chart:** A line chart visualizing how Total Profit (Y-axis) decays as Latency (X-axis) increases.
3. **Top Opportunities:** A list of the Top 5 most profitable ISINs (at 0 latency). **Sanity check these results**—do they look real?

### 1. The "Money Table"

In [ ]:
# Your code here


### 2. The Decay Chart

In [ ]:
# Your code here


### 3. Top Opportunities

In [ ]:
# Your code here


## Grading Rubric (Max 10 Points)

- **5-6 Points (Baseline):** The code runs, correctly calculates the consolidated tape, identifies Bid > Ask opportunities, and estimates theoretical (0 latency) profit.

- **7-8 Points (Robust):** The simulation accurately models latency (using strict time-lookups) and strictly adheres to the vendor's data quality specs.

- **9-10 Points (Expert):** You demonstrate deep understanding of market microstructure. You handle **Market Status** correctly to avoid fake signals, identify anomalies in the instrument list, and handle edge cases around Market Open/Close.